# Example of ChatBot
> In this project I will show how you can easy create your ouwn ***Q/A*** customer support chat bot. Also will provide some helpfull function that could be usefull in your projects, or tasks win AI field.

In [1]:
#Importing necessary packages
from google.cloud import bigquery
import pandas as pd 
import numpy as np
import requests
import datetime
import time
from google.oauth2.service_account import Credentials
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch, Chroma
from langchain.document_loaders import TextLoader, PyPDFLoader, WebBaseLoader
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
import jsonlines
import  jpype     
import  asposecells     
jpype.startJVM() 
from asposecells.api import Workbook
from pydantic import BaseModel, Field
import os
import openai
import sys
sys.path.append('../..')
from dotenv import load_dotenv, find_dotenv
from langchain.utils.openai_functions import convert_pydantic_to_openai_function 
import panel as pn
import param
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser,JsonKeyOutputFunctionsParser
from typing import Optional, List
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.tools import tool
import wikipedia
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [2]:
#Connect to OpenAI 
_ = load_dotenv(find_dotenv()) #read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:

#Loading access key
key_path = "serv_acc_key.json"
#Create credentials object
credentials = Credentials.from_service_account_file(
    key_path,
    scopes = ['https://www.googleapis.com/auth/cloud-platform']
)
if credentials.expired:
    credentials.refresh(Request())

# Part 1: Data Exctraction
- 1.1. The first example will show how to extract data from Google Cloud.
- 1.2. Second, how you can fetch data from Wikipedia depends on your request.

### 1.1. Exctraction from Google Cloud

In [4]:
#Connect to BigQuery
PROJECT_ID = 'praxis-window-402615'
def run_bq_query(sql):

    # Create BQ client
    bq_client = bigquery.Client(project = PROJECT_ID, 
                                credentials = credentials)

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, 
                                         use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, 
                                    job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")
    return df


In [5]:
#define list of programming language tags we want to query
language_list = ["python","html","r","css"]
so_df = pd.DataFrame()

for language in language_list:
    
    print(f"generating {language} dataframe")
    
    query = f"""
    SELECT
        CONCAT(q.title, q.body) as question,
        a.body AS answer
    FROM
        `bigquery-public-data.stackoverflow.posts_questions` q
    JOIN
        `bigquery-public-data.stackoverflow.posts_answers` a
    ON
        q.accepted_answer_id = a.id
    WHERE 
        q.accepted_answer_id IS NOT NULL AND 
        REGEXP_CONTAINS(q.tags, "{language}") AND
        a.creation_date >= "2020-01-01"
    LIMIT 
        500
    """

    
    language_df = run_bq_query(query)
    language_df["category"] = language
    so_df = pd.concat([so_df, language_df], 
                      ignore_index = True) 
    

generating python dataframe
Finished job_id: 009377c2-fee6-4c9d-9f09-b4877f241eac
generating html dataframe
Finished job_id: 97176d4a-d0f8-42ca-ba7e-3dfb8e1747fd
generating r dataframe
Finished job_id: e07b1c04-7ede-4646-9fa5-543e27936a8b
generating css dataframe
Finished job_id: 61144b45-64bc-4457-b41e-c76dd48c7192


In [6]:
#Checking what we have
display(so_df)

,question,answer,category
0,How can I distribute member to it's role on py...,"<p>Try:</p>\n<pre class=""lang-py prettyprint-o...",python
1,Pydantic Inherited Class validation<p>I have 2...,<blockquote>\n<p>The problem is that I can't s...,python
2,Why I can't get the shortest root to leaf heig...,<p>First some issues to make your code runnabl...,python
3,How to create from a 2-column DataFrame a Data...,<p>Try this:</p>\n<pre><code>ndf = (df.set_ind...,python
4,discord.py accept - if clause: .MissingRequire...,<p>Discord requires the parameter to be set un...,python
...,...,...,...
1995,css grid grows over 100% width with fr but not...,<p><strong>Solution</strong>:</p>\n\n<p>Settin...,css
1996,Javascript background Blur/Popup box transitio...,<p>Change your <code>toggle()</code> function ...,css
1997,How to change Bootstrap navbar color on scroll...,"<blockquote>\n<p>I Just tested this, and it wo...",css
1998,Fadeout effect is not working using wow animat...,<p>You can add <code>animation-fill-mode: forw...,css


### 1.2. Fetching from Wikipedia

In [59]:
#Creating a function that will fetch relevant information from wikipedia depends on user input
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""#Creating description that our llm will read
    page_titles = wikipedia.search(query)#Scrap relevant information from wikipedia
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)#Creating a page
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")#Appending all fetched info to our variable
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)


In [64]:
#Checking if all working properly
search_wikipedia({"query": "Machine Learning"})

'Page: Machine learning\nSummary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can effectively generalize and thus perform tasks without explicit instructions. Recently, generative artificial neural networks have been able to surpass many previous approaches in performance. Machine learning approaches have been applied to large language models, computer vision, speech recognition, email filtering, agriculture and medicine, where it is too costly to develop algorithms to perform the needed tasks.The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis through unsupervised learning.ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statis

> Now that you have two functions that can be used for data extraction. Next, you need to pass this data into Vectorstore or Vectordb or you can simply save it in JSON or pdf file on your computer. Here is one example of how you can fast and easily store your data on your computer

In [44]:
#Common way of storing your data to json format
df = so_df.to_dict()
with jsonlines.open(f'qa.json','w') as writer:
    writer.write_all(df)

In [45]:
#Converting our Json file to PD
workbook = Workbook("qa.json")
workbook.save("qa.pdf")
jpype.shutdownJVM()

In [ ]:
#Function that will load your pdf file and create a Vector index store to which you could connect your llm
def load_db(file,chain_type, k):
    #load documents
    loader=PyPDFLoader(file)
    documents = loader.load()
    #split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    #define embedding
    embeddings = OpenAIEmbeddings()
    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs,embeddings)
    #define retriever
    retriever = db.as_retriever(search_type ="similarity",search_kwargs = {"k":k})
    #Create a chatbot chain. Memory is managed externally
    qa = ConversationalRetrievalChain.from_llm(
        llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0),
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents = True,
        return_generated_question = True,
    )

    return qa

>**Conclusion:** So now after you get two example how you can on a simple wat get a necessary data work with your chat let's show several tools that can be helpful for you, when you will build your customer chatbots

# Part 2: Function tools
 > Also you can create a different function depending on what kind of question the user will ask
- Here I will create two different function 
- And show you how with the help of **pydantic** library you can build your function in a faster and simplest way

In [3]:
#This function will fetch all relevant information about machine learning
class ML_tools_search(BaseModel):
    """Call this with a Machine learning code to extract any relevant information about Machine learning"""
    machine_learning: str = Field(description="machine learning code  to get information about")
ML_function = convert_pydantic_to_openai_function(ML_tools_search)
ML_function

{'name': 'ML_tools_search',
 'description': 'Call this with an Machine learning code to extract any relevant information about Machine learning',
 'parameters': {'title': 'ML_tools_search',
  'description': 'Call this with an Machine learning code to extract any relevant information about Machine learning',
  'type': 'object',
  'properties': {'machine_learning': {'title': 'Machine Learning',
    'description': 'machine learning code  to get information about',
    'type': 'string'}},
  'required': ['machine_learning']}}

In [9]:
#Second function will fetch all relevant information about artist
class ArtistSearch(BaseModel):
    """Call this to get the name of songs by a particular artist"""
    artist_name: str= Field(description="name of artist to look up")
    n: int = Field(description="number of results")

artist_function = convert_pydantic_to_openai_function(ArtistSearch)
artist_function

{'name': 'ArtistSearch',
 'description': 'Call this to get the name of songs by a particular artist',
 'parameters': {'title': 'ArtistSearch',
  'description': 'Call this to get the name of songs by a particular artist',
  'type': 'object',
  'properties': {'artist_name': {'title': 'Artist Name',
    'description': 'name of artist to look up',
    'type': 'string'},
   'n': {'title': 'N', 'description': 'number of results', 'type': 'integer'}},
  'required': ['artist_name', 'n']}}

In [10]:
#Insert all our function to one variable
functions = [
    ML_function,
    artist_function
]
#Cre
model = ChatOpenAI()
model_with_functions = model.bind(functions=functions)
model_with_functions.invoke("What type of ML most popular?")

AIMessage(content='The most popular type of machine learning currently is deep learning, which is a subset of artificial intelligence. Deep learning algorithms are designed to simulate the workings of the human brain and are particularly effective in tasks such as image and speech recognition, natural language processing, and recommendation systems.')

In [11]:
model_with_functions.invoke("what are the most popular songs by Bon Jovi?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ArtistSearch', 'arguments': '{\n  "artist_name": "Bon Jovi",\n  "n": 5\n}'}})

>**Conclusion:** So By the previous line, you can see that when we called our model to find information about our artist the model called for our **ArtistSearch** function this is one example of using Functions depending on what kind of operations users ask of your model. Let's create a workable chain that will provide reasons and answer with help of functions that we have created previously

In [22]:
#Exaple of using prompt + model + output 
prompt = ChatPromptTemplate.from_messages([
    ("system","Your are assistant, and you need provide any relavant information"),
    ("user","{input}")
]
)
#Creating a LLM chain and provide output in Json format
chain = prompt | model_with_functions | JsonOutputFunctionsParser()
chain.invoke({"input":"What type of musing singing Bon Jovi?"})

{'artist_name': 'Bon Jovi', 'n': 5}

In [72]:
# We can also use extraction function, that will extract all relant information from a text/file
# Here better to use JsonOutputKeyFunctionParser that will look only for relevant key inside text
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])
#Creating a function that will work as a person information extracter
class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")
    gender: str = Field(description="person's gender")

class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")

#Creating necessary function by pydantic
extraction_functions = [convert_pydantic_to_openai_function(Information)]
#Applying function to model
extraction_model = model.bind(functions=extraction_functions, function_call={"name": "Information"})
#Building a Chain
extraction_chain = prompt | extraction_model | JsonOutputFunctionsParser()
#Calling chain
extraction_chain.invoke({"input":"Joe is 30, his mom is Martha"})

{'people': [{'name': 'Joe', 'age': 30, 'gender': 'male'},
  {'name': 'Martha', 'age': None, 'gender': None}]}

### Put all together:
- Now I will all these techniques together in a real-world example
- In this example, I will extract information from a blog post and provide a necessary format such as JSON or dictionary

In [31]:
#First loading data 
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
documents = loader.load()#download data from website
#fetching only  page content information
doc = documents[0]
page_content = doc.page_content[:1000]
print(page_content[:1000])








LLM Powered Autonomous Agents | Lil'Log







































Lil'Log






















Posts




Archive




Search




Tags




FAQ




emojisearch.app









      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


 


Table of Contents



Agent System Overview

Component One: Planning

Task Decomposition

Self-Reflection


Component Two: Memory

Types of Memory

Maximum Inner Product Search (MIPS)


Component Three: Tool Use

Case Studies

Scientific Discovery Agent

Generative Agents Simulation

Proof-of-Concept Examples


Challenges

Citation

References





Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general

In [37]:
#Creating function that will provide a summary of the content,language and keywords of the content
class Overview(BaseModel):
    """Overview of a section of text"""
    summary: str= Field(description="Provide a concise summary of the content.")
    language: str = Field(description="Provide the language that the content is written in.")
    kewords: str = Field(description="Provide keywords related to the content")

overview_function = [
    convert_pydantic_to_openai_function(Overview)
]
over_model = model.bind(
    functions = overview_function,
    function_call = {"name":"Overview"}

)
#Build prompt
prompt = ChatPromptTemplate.from_messages([
    ("system","Extract the relenvant information, if not explicity provided do not quess. Exctract partial info"),
    ("human","{input}")
])
extract_chain = prompt | over_model | JsonOutputFunctionsParser()

#Calling our extractiong chain
extract_chain.invoke({"input":page_content})

{'summary': 'The blog post discusses the concept of building autonomous agents using LLM (large language model) as its core controller. It mentions proof-of-concept demos and the potential of LLM beyond generating written content.',
 'language': 'English',
 'keywords': 'LLM, autonomous agents, planning, memory, tool use, proof-of-concept demos'}

In [50]:
#Also you can split your documents into chunck and exctrac information from there
#You can add to your chain some data preparation methods that will give you opurtunity to exctract different type of informations
#And you can combine two different chains and get final result
# Here I propose to create a new function that will fetch relevant information from each papers inside our documents
class Paper(BaseModel):
    """Information about papers mentioned."""
    title: str
    author: Optional[str]


class Info(BaseModel):
    """Information to extract"""
    papers: List[Paper]


paper_extraction_function = [
    convert_pydantic_to_openai_function(Info)
]
extraction_model = model.bind(
    functions=paper_extraction_function, 
    function_call={"name":"Info"}
)


#Creating prompt template
template = """A article will be passed to you. Extract from it all papers that are mentioned by this article. 

Do not extract the name of the article itself. If no papers are mentioned that's fine - you don't need to extract any! Just return an empty list.

Do not make up or guess ANY extra information. Only extract what exactly is in the text."""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", "{input}")
])
#First we will create Exctraction chain
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="papers")

#Splitting text into a chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_overlap =0)
#Now we need to combine our splitting text(chunks) 
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list
#Will prepera our text into right format (e.g. "Input":"text")
prep = RunnableLambda(
    lambda x: [{"input": doc} for doc in text_splitter.split_text(x)]
)
#We call map in exctract_chain beause we will use several operations inside 
chain = prep | extraction_chain.map() | flatten

In [51]:
chain.invoke(doc.page_content)

[{'title': 'Chain of thought (CoT; Wei et al. 2022)', 'author': ''},
 {'title': 'Tree of Thoughts (Yao et al. 2023)', 'author': ''},
 {'title': 'LLM+P (Liu et al. 2023)', 'author': ''},
 {'title': 'ReAct (Yao et al. 2023)', 'author': ''},
 {'title': 'Reflexion (Shinn & Labash 2023)', 'author': ''},
 {'title': 'Reflexion framework', 'author': 'Shinn & Labash'},
 {'title': 'Chain of Hindsight', 'author': 'Liu et al. 2023'},
 {'title': 'Algorithm Distillation', 'author': 'Laskin et al. 2023'},
 {'title': 'Algorithm Distillation', 'author': 'Laskin et al. 2023'},
 {'title': 'ED (expert distillation)', 'author': ''},
 {'title': 'RL^2', 'author': 'Duan et al. 2017'},
 {'title': 'MRKL: Modular Reasoning, Knowledge and Language',
  'author': 'Karpas et al. 2022'},
 {'title': 'TALM: Tool Augmented Language Models',
  'author': 'Parisi et al. 2022'},
 {'title': 'Toolformer', 'author': 'Schick et al. 2023'},
 {'title': 'HuggingGPT', 'author': 'Shen et al. 2023'},
 {'title': 'API-Bank (Li et al. 2

>**Conclusion:**  So with the help of previous examples and tools, you can simply fetch a lot of useful information from ur data and save it or pass it to the LLM or your vector database, which can provide users with a good answer or generate a necessary answer or provide reasonable information to them

## Part 3. Creating a Testing Chat bot
- In this step I will show you how you can create a simple chatbot with some additional tools inside, that can be used for different operations depending on what kind of query passes to the model from the user.

In [ ]:
#Calling for extension
pn.extension()


In [ ]:

#Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitute of the location to fetch weather data for")
    longitude: float = Field(..., description="Longtitude of the location to fetch weather data for ")


#Creating a function the will take parameters from the previous function as inputs and provide a necessary output based on these inputs
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    #Parameters for the request
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly":"temperature_2m",
        "forecast_days":1,
    }
    #Make the request
    response = requests.get(BASE_URL,params=params)
     
    if response.status_code == 200:
       results = response.json()
    else:
        raise Exception(f"Api Requst failed with status code: {response.status_code}")
    
    current_utc_time = datetime.datetime.utcnow()#fetching present time
    #converting to a necessary format
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z','+00:00')) for time_str in results['hourly']['time']]
    #creating list of time/temperature
    temperature_list = results['hourly']['temperature_2m']

    #Finding a more nearest temperature for our present time from our previous created list
    closest_time_index = min(range(len(time_list)), key = lambda i: abs(time_list[i] - current_utc_time))
    #scrapping temperature 
    current_temperature = temperature_list[closest_time_index]

    return print(f"The current temperature is {current_temperature} C")

In [ ]:
#Creating a function that will fetch relevant information from wikipedia depends on user input
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""#Creating description that our llm will read
    page_titles = wikipedia.search(query)#Scrap relevant information from wikipedia
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)#Creating a page
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")#Appending all fetched info to our variable
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)


In [73]:
#You can also create your own function 
@tool
def create_your_own(quesry: str) -> str:
    """This function can do whatever you would like once you fill it in"""
    print(type(query))
    return query[::-1]


In [ ]:
#Assigned tools 
tools = [get_current_temperature,search_wikipedia,create_your_own]

In [46]:
#Creating class that will work for us like a brain to our chatbot
pn.extension()

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 
    

In [16]:
#Implementing frontend steps of our ChatBot 
b = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')#Line of entering user input

conversation = pn.bind(cb.convchain, inp)

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)#Assigned name of chatbot and column from where you will see all input and output text

# Overal Conclusion:
> In this project I showed you several tools:
- 1) Techniques of extraction and preparing your data to be ready to go to your LLM or another Machine learning algorithm you can work with
- 2) Secondary by creating functions and putting them into your model, you can provide the opportunity to determine which function is better for you depending on what kind of user query(input) will be. This is helpful when you want to build a more complex chatbot that can work not only on simple tasks such as Q/A but also provide summaries, analysis, or some operations on the data that you have.
- 3) In the last part of this project. I created a chatbot with a simple interface by using **pn** library and showed how it can work in your local **Jupiter, vs code etc.** 